In [2]:
import pandas as pd
import numpy as np
import phonenumbers as ph
import pycountry as pc
from fuzzywuzzy import fuzz
import regex
import json

In [3]:
#Reading data into pandas dataframes
fb_df = pd.read_csv('../../datasets/input/facebook_dataset.csv', delimiter=",", quotechar='"', escapechar='\\' ,dtype=str)
g_df = pd.read_csv('../../datasets/input/google_dataset.csv', delimiter=",", quotechar='"', escapechar='\\' ,dtype=str)
w_df = pd.read_csv('../../datasets/input/website_dataset.csv', delimiter=";", quotechar='"',dtype=str)

In [4]:
#Filling in null values
fb_df = fb_df.replace(['NaN','nan','NAN'], None)
g_df = g_df.replace(['NaN','nan','NAN'], None)
w_df = w_df.replace(['NaN','nan','NAN'], None)

In [5]:
#Creating an identifier column to let-us know the source of the data
fb_df['identifier'] = fb_df.index.map(lambda x: str(x) + '-facebook') 
g_df['identifier'] = g_df.index.map(lambda x: str(x) + '-google')
w_df['identifier'] = w_df.index.map(lambda x: str(x) + '-website')

fb_col_dy = set(fb_df.columns)
g_col_dy = set(g_df.columns)
w_col_dy = set(w_df.columns)

common_columns_fb_g = fb_col_dy.intersection(g_col_dy)
common_columns_fb_w = fb_col_dy.intersection(w_col_dy)
common_columns_g_w = g_col_dy.intersection(w_col_dy)
common_columns_all =  fb_col_dy.intersection(g_col_dy).intersection(w_col_dy)

print(f'Common columns fb-google  : {common_columns_fb_g}')
print(f'Common columns fb-web     : {common_columns_fb_w}')
print(f'Common columns google-web : {common_columns_g_w}')
print(f'Common columns all        : {common_columns_all}')

Common columns fb-google  : {'address', 'phone', 'domain', 'country_code', 'name', 'country_name', 'region_code', 'zip_code', 'identifier', 'city', 'region_name', 'phone_country_code'}
Common columns fb-web     : {'phone', 'identifier'}
Common columns google-web : {'phone', 'identifier'}
Common columns all        : {'phone', 'identifier'}


In [6]:
#Renaming columns to be the same where there is same content
w_df.rename(columns={'root_domain':'domain',
                    'main_city':'city',
                    'main_country':'country_name',
                    'main_region':'region',
                    's_category':'category'}, inplace=True)

# g_df.rename(columns={'raw_address':'address'
#                     }, inplace=True)

fb_df.rename(columns={'categories':'category'
                    }, inplace=True)

In [7]:
fb_col_dy = set(fb_df.columns)
g_col_dy = set(g_df.columns)
w_col_dy = set(w_df.columns)

common_columns_fb_g = fb_col_dy.intersection(g_col_dy)
common_columns_fb_w = fb_col_dy.intersection(w_col_dy)
common_columns_g_w = g_col_dy.intersection(w_col_dy)
common_columns_all =  fb_col_dy.intersection(g_col_dy).intersection(w_col_dy)

print(f'Common columns fb-google  : {common_columns_fb_g}')
print(f'Common columns fb-web     : {common_columns_fb_w}')
print(f'Common columns google-web : {common_columns_g_w}')
print(f'Common columns all        : {common_columns_all}')

Common columns fb-google  : {'address', 'phone', 'domain', 'country_code', 'name', 'country_name', 'category', 'region_code', 'zip_code', 'identifier', 'city', 'region_name', 'phone_country_code'}
Common columns fb-web     : {'phone', 'domain', 'country_name', 'category', 'identifier', 'city'}
Common columns google-web : {'phone', 'domain', 'country_name', 'category', 'identifier', 'city'}
Common columns all        : {'phone', 'domain', 'country_name', 'category', 'identifier', 'city'}


In [8]:
# Function to format and validate phone numbers
def format_and_validate_phone_number(phone_number):
    try:
        if not phone_number.startswith('+'):
            phone_number = "+"+phone_number    
        parsed_number = ph.parse(phone_number, None)
        formatted_number = ph.format_number(parsed_number, ph.PhoneNumberFormat.INTERNATIONAL)
        return formatted_number if ph.is_valid_number(parsed_number) else None
    except ph.phonenumberutil.NumberParseException:
        return None
    
def parse_country_name_to_code(country_name):
    try:
        country = pc.countries.get(name=country_name)
        if country is not None:
            return str.lower(country.alpha_2)
        else:
            return None
    except LookupError:
        return None

def normalise_comapny_name(company_name):
    # Define a list of company types to remove
    company_types = ['Ltd', 'Inc', 'Corp', 'LLC', 'Ltd.', 'Inc.', 'Corp.', 'LLC.', 'Co', 'AG', 'PLC', 'SA', 'SRL', 'NV', 'Pty', 'AB', 'BV', 'MD']

   # Remove special characters from the company name
    company_name = regex.sub(r'\p{P}', '', company_name) #removing special characters
    company_name = regex.sub(r'\+[0-9]+|\<|\>|°|^[0-9]+$', '',company_name) # removingspecial characters not removed previously
    company_name = regex.sub(r'[^\w\s]', '', company_name) # removing emoticons

    # Use regular expression to match and remove company types
    pattern = regex.compile(r'\b(?:{})\b'.format('|'.join(map(regex.escape, company_types))), regex.IGNORECASE)
    company_name = pattern.sub('', company_name)

    # Remove leading/trailing whitespaces and reduce multiple spaces to a single space
    company_name = regex.sub(r'\s+', ' ', company_name).strip()
    if (str.upper(company_name) == 'NAN' or company_name == ''):
        return np.nan
    else:
        return str.upper(company_name)

def clean_city(string):
    if pd.isnull(string):
        return None
    elif isinstance(string, str):
        string = regex.sub( r"[^\w\s]", '', string)  # Remove special characters
        string = string.strip()  # Remove trailing and leading spaces
        return string
    else:
        return string

def create_json_address(df):
    json_addresses = []
    unique_addresses = set()
    for _, row in df.iterrows():
        address = {
            'id': row['identfier'],
            'phone': row['phone_parsed'],
            # 'country': row['country_code'],
            'city': row['city'],
            'zip_code': row['zip_code'],
            'raw_address': row['address']
        }
        address_str = json.dumps(address)
        if address_str not in unique_addresses:
            json_addresses.append(address)
            unique_addresses.add(address_str)
        
    return json.dumps(json_addresses)

def calculate_company_name_fuzzy_score(row):
    company_name_x = str(row['company_name_norm_x'])  # Convert to string
    company_name_y = str(row['company_name_norm_y'])  # Convert to string
    
    # Handle NaN values
    if company_name_x == 'nan' or company_name_y == 'nan':
        return 0
    
    return fuzz.ratio(company_name_x, company_name_y)

def check_category_inclusion(row):
    result = np.nan
    category_x = str(row['category_x']) # Convert to string
    category_y = str(row['category_y']) # Convert to string
    if (category_x == 'nan' and category_y == 'nan') or (category_x != 'nan' and category_y == 'nan'):
        result = 0
    else:
        if category_x == 'nan' and category_y != 'nan':
            result = 1
        else:
            if category_y in category_x:
                result = 0
            else:
                result = 1
    return result

## Cleaning Data

In [9]:
# fix the data types

fb_df = fb_df.astype(str)
g_df = g_df.astype(str)
w_df = w_df.astype(str)

#Checking domain colums to see if it contains a valid domain string
# Regex pattern for domain validation
regex_pattern = r'^[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

fb_df['valid_domain'] = np.where(fb_df['domain'].str.match(regex_pattern),0,1) # 0 - true, 1 - false
g_df['valid_domain'] = np.where(g_df['domain'].str.match(regex_pattern),0,1) # 0 - true, 1 - false
w_df['valid_domain'] = np.where(w_df['domain'].str.match(regex_pattern),0,1) # 0 - true, 1 - false

In [14]:
fb_df['valid_domain'].value_counts()

0    72009
1        1
Name: valid_domain, dtype: int64

In [15]:
g_df['valid_domain'].value_counts()

0    356519
1         1
Name: valid_domain, dtype: int64

In [16]:
w_df['valid_domain'].value_counts()

0    72004
1       14
Name: valid_domain, dtype: int64

In [17]:
#Checking phone-numbers
fb_df['phone_parsed'] = fb_df['phone'].apply(format_and_validate_phone_number)
g_df['phone_parsed'] = g_df['phone'].apply(format_and_validate_phone_number)
w_df['phone_parsed'] = w_df['phone'].apply(format_and_validate_phone_number)

In [18]:
#Normalising Company Names
fb_df['company_name_norm'] = fb_df['name'].apply(normalise_comapny_name)
g_df['company_name_norm'] = g_df['name'].apply(normalise_comapny_name)
w_df['company_name_norm'] = w_df['legal_name'].apply(normalise_comapny_name)

## Enriching Data

In [19]:
#getting country code for websites df
w_df['country_code'] = w_df['country_name'].apply(parse_country_name_to_code)

#adding missing columns with null for the websites df
w_df['address'] = None
w_df['zip_code'] = None


In [20]:
#Checking the common columns 
common_columns_all =  fb_df.columns.intersection(g_df.columns).intersection(w_df.columns)

print(f'Commnon columns: {common_columns_all}')

Commnon columns: Index(['domain', 'address', 'category', 'city', 'country_code', 'country_name',
       'phone', 'zip_code', 'identifier', 'valid_domain', 'phone_parsed',
       'company_name_norm'],
      dtype='object')


## Creating Normalised dataset

In [21]:
normalised_columns = ['identifier','company_name_norm','country_code','phone_parsed','domain','city','zip_code','category','address']

fb_norm_df = fb_df[fb_df['valid_domain']==0][normalised_columns].copy()
fb_norm_df = fb_norm_df.replace(['NaN','nan','NAN'], None)

g_norm_df = g_df[g_df['valid_domain']==0][normalised_columns].copy()
g_norm_df = g_norm_df.replace(['NaN','nan','NAN'], None)

w_norm_df = w_df[w_df['valid_domain']==0][normalised_columns].copy()
w_norm_df = w_norm_df.replace(['NaN','nan','NAN'], None)


In [ ]:
fb_norm_df.to_parquet('../../datasets/working/fb_norm.parquet')
g_norm_df.to_parquet('../..datasets/working/g_norm.parquet')
w_norm_df.to_parquet('../..datasets/working/w_norm.parquet')

### Merging all data
- Merging data into one big dataset to start enriching and deduplication
- Ideally would be good to merge on infomation that is consistent, like domain, zip code, phone number, or combinations between those.
- Mergin on domain is only possible between facebook and webistes datasets as google has common social media or industry platform domains.
- Before mergin some more data cleanup will be done in order to maximize the joining success.

In [22]:
g_norm_df['domain'].value_counts()

facebook.com                      71983
postoffice.co.uk                   6010
instagram.com                      5647
ihg.com                            4356
marriott.com                       3979
                                  ...  
streatsideeatery.com                  1
slsi.ca                               1
maplemoonwebdesign.com                1
dentalperfections.co.uk               1
aspenapplianceandfurniture.com        1
Name: domain, Length: 72009, dtype: int64

## Flagging Data

In [23]:
null_mask = w_norm_df['company_name_norm'].isnull()
w_total = len(w_norm_df)
w_null_company = len(w_norm_df[null_mask])
w_nnull_company = len(w_norm_df[~null_mask])

null_mask = g_norm_df['company_name_norm'].isnull()
g_total = len(g_norm_df)
g_null_company = len(g_norm_df[null_mask])
g_nnull_company = len(g_norm_df[~null_mask])

null_mask = fb_norm_df['company_name_norm'].isnull()
fb_total = len(fb_norm_df)
fb_null_company = len(fb_norm_df[null_mask])
fb_nnull_company = len(fb_norm_df[~null_mask])

print(f'Websites company name consistency: \n\t - Null Values    :  {w_null_company} [ {round((w_null_company/w_total)*100,2)} %] \n\t - Not Null Values:  {w_nnull_company} [ {round((w_nnull_company/w_total)*100,2)} %]')
print(f'Facebook company name consistency: \n\t - Null Values    :      {fb_null_company} [ {round((fb_null_company/fb_total)*100,2)} %] \n\t - Not Null Values:  {fb_nnull_company} [ {round((fb_nnull_company/fb_total)*100,2)} %]')
print(f'Google company name consistency  : \n\t - Null Values    :     {g_null_company} [ {round((g_null_company/g_total)*100,2)} %] \n\t - Not Null Values: {g_nnull_company} [ {round((g_nnull_company/g_total)*100,2)} %]')

Websites company name consistency: 
	 - Null Values    :  40035 [ 55.6 %] 
	 - Not Null Values:  31969 [ 44.4 %]
Facebook company name consistency: 
	 - Null Values    :      3 [ 0.0 %] 
	 - Not Null Values:  72006 [ 100.0 %]
Google company name consistency  : 
	 - Null Values    :     41 [ 0.01 %] 
	 - Not Null Values: 356478 [ 99.99 %]


In [24]:
#Getting all companies we have a name for
c_fb_norm_df = fb_norm_df[fb_norm_df['company_name_norm'].notnull()].copy()
c_fb_norm_df = c_fb_norm_df.reset_index(drop=True)

c_g_norm_df = g_norm_df[g_norm_df['company_name_norm'].notnull()].copy()
c_g_norm_df = c_g_norm_df.reset_index(drop=True)

c_w_norm_df = w_norm_df[w_norm_df['company_name_norm'].notnull()].copy()
c_w_norm_df = c_w_norm_df.reset_index(drop=True)

full_df = pd.concat( [c_fb_norm_df, c_g_norm_df ,c_w_norm_df], ignore_index=True)
full_df = full_df.replace(['NaN','nan','NAN'],None)
full_df.shape

(460453, 9)

In [25]:
#checking city for special characters
full_df['city'] = full_df['city'].apply(clean_city)
full_df = full_df.replace(['NaN','nan','NAN'],None)
full_df.to_parquet('../../datasets/working/full_df.parquet')

In [31]:
total_records = len(full_df)
null_country  = len(full_df[full_df['country_code'].isnull()])
null_domain  = len(full_df[full_df['domain'].isnull()])
null_city = len(full_df[full_df['city'].isnull()])
null_zip = len(full_df[full_df['zip_code'].isnull()])
null_phones = len(full_df[full_df['phone_parsed'].isnull()])
null_category = len(full_df[full_df['category'].isnull()])
null_address = len(full_df[full_df['address'].isnull()])

print(f'Total records                 : {total_records}    [100.00 %]')
print(f'Total records no country      :  {null_country}    [ {round((null_country/total_records)*100,2)} %]')
print(f'Total records no domain       :      {null_domain}    [  {round((null_domain/total_records)*100,2)}  %]')
print(f'Total records no city         :  {null_city}    [  {round((null_city/total_records)*100,2)} %]')
print(f'Total records no zip          : {null_zip}    [  {round((null_zip/total_records)*100,2)} %]')
print(f'Total records no phone_parsed :  {null_phones}    [ {round((null_phones/total_records)*100,2)} %]')
print(f'Total records no category     :  {null_category}    [ {round((null_category/total_records)*100,2)} %]')
print(f'Total records no address      :  {null_address}    [ {round((null_address/total_records)*100,2)} %]')

Total records                 : 460453    [100.00 %]
Total records no country      :  68954    [ 14.98 %]
Total records no domain       :      0    [  0.0  %]
Total records no city         :  75508    [  16.4 %]
Total records no zip          : 151015    [  32.8 %]
Total records no phone_parsed :  63640    [ 13.82 %]
Total records no category     :  68503    [ 14.88 %]
Total records no address      :  72535    [ 15.75 %]


## End part 1

In [39]:
#Trying to understand the google dataset
total_records = len(g_norm_df)
null_country  = len(g_norm_df[g_norm_df['country_code'].isnull()])
null_domain  = len(g_norm_df[g_norm_df['domain'].isnull()])
null_city = len(g_norm_df[g_norm_df['city'].isnull()])
null_zip = len(g_norm_df[g_norm_df['zip_code'].isnull()])
null_phones = len(g_norm_df[g_norm_df['phone_parsed'].isnull()])
null_category = len(g_norm_df[g_norm_df['category'].isnull()])
null_address = len(g_norm_df[g_norm_df['address'].isnull()])
null_company_name = len(g_norm_df[g_norm_df['company_name_norm'].isnull()])

print(f'Total records                 : {total_records}    [100.00 %]')
print(f'Total records no country      :  {null_country}    [ {round((null_country/total_records)*100,2)} %]')
print(f'Total records no domain       :      {null_domain}    [  {round((null_domain/total_records)*100,2)}  %]')
print(f'Total records no city         :  {null_city}    [ {round((null_city/total_records)*100,2)} %]')
print(f'Total records no zip          :  {null_zip}    [ {round((null_zip/total_records)*100,2)} %]')
print(f'Total records no phone_parsed :  {null_phones}    [  {round((null_phones/total_records)*100,2)} %]')
print(f'Total records no category     :  {null_category}    [ {round((null_category/total_records)*100,2)} %]')
print(f'Total records no address      :  {null_address}    [  {round((null_address/total_records)*100,2)} %]')
print(f'Total records no address      :     {null_company_name}    [  {round((null_company_name/total_records)*100,2)} %]')

Total records                 : 356519    [100.00 %]
Total records no country      :  52435    [ 14.71 %]
Total records no domain       :      0    [  0.0  %]
Total records no city         :  45518    [ 12.77 %]
Total records no zip          :  83120    [ 23.31 %]
Total records no phone_parsed :  32685    [  9.17 %]
Total records no category     :  51650    [ 14.49 %]
Total records no address      :  25968    [  7.28 %]
Total records no address      :     41    [  0.01 %]


## Move to next notebook

In [31]:
%%time
agg_data = full_df.groupby(['company_name_norm','country_code','domain'])\
                     .apply(lambda df: pd.Series({
                                          'rows_count': len(df),
                                          'categories': '|'.join(df['category'].unique()),
                                          'addresses': create_json_address(df)
                                                        })).reset_index()

CPU times: user 3min, sys: 1.8 s, total: 3min 2s
Wall time: 3min 3s


In [18]:
w_by_domain = w_df.groupby(['root_domain'])\
                .apply(lambda df: pd.Series({
                            'rows_count': len(df),
                            'legal_names': '||'.join(df['legal_name'].unique()),
                            'countries': '||'.join(df['main_country'].unique())
                                            })).reset_index()

w_multiple_companies = w_by_domain[w_by_domain['rows_count']>1]
w_single_companies = w_by_domain[w_by_domain['rows_count']==1]

print(f'w_multiple_companies: {w_multiple_companies}')
print(f'w_single_companies: {w_single_companies}')
# to normalise:
# - main_country
# - main_region

In [21]:
g_by_domain = g_df.groupby(['domain'])\
                .apply(lambda df: pd.Series({
                            'rows_count': len(df),
                            'legal_names': '||'.join(df['name'].unique()),
                            'countries': '||'.join(df['country_code'].unique())
                                            })).reset_index()

g_multiple_companies = g_by_domain[g_by_domain['rows_count']>1].count()
g_single_companies = g_by_domain[g_by_domain['rows_count']==1].count()

print(f'g_multiple_companies: {g_multiple_companies}')
print(f'g_single_companies: {g_single_companies}')

g_multiple_companies: domain         15291
rows_count     15291
legal_names    15291
countries      15291
dtype: int64
g_single_companies: domain         56719
rows_count     56719
legal_names    56719
countries      56719
dtype: int64
